Given financial client data, let's try to predict whether a given client will go bankrupt or not.

We will use a variety of classification models to make our prediction.

**Getting started**

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('max_columns', None)

import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
data = pd.read_csv('/content/bankruptcy prediction.csv', header=0)

In [ ]:
data

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,Continuous interest rate (after tax),Operating Expense Rate,Research and development expense rate,Cash flow rate,Interest-bearing debt interest rate,Tax rate (A),Net Value Per Share (B),Net Value Per Share (A),Net Value Per Share (C),Persistent EPS in the Last Four Seasons,Cash Flow Per Share,Revenue Per Share (Yuan ¥),Operating Profit Per Share (Yuan ¥),Per Share Net profit before tax (Yuan ¥),Realized Sales Gross Profit Growth Rate,Operating Profit Growth Rate,After-tax Net Profit Growth Rate,Regular Net Profit Growth Rate,Continuous Net Profit Growth Rate,Total Asset Growth Rate,Net Value Growth Rate,Total Asset Return Growth Rate Ratio,Cash Reinvestment %,Current Ratio,Quick Ratio,Interest Expense Ratio,Total debt/Total net worth,Debt ratio %,Net worth/Assets,Long-term fund suitability ratio (A),Borrowing dependency,Contingent liabilities/Net worth,Operating profit/Paid-in capital,Net profit before tax/Paid-in capital,Inventory and accounts receivable/Net value,Total Asset Turnover,Accounts Receivable Turnover,Average Collection Days,Inventory Turnover Rate (times),Fixed Assets Turnover Frequency,Net Worth Turnover Rate (times),Revenue per person,Operating profit per person,Allocation rate per person,Working Capital to Total Assets,Quick Assets/Total Assets,Current Assets/Total Assets,Cash/Total Assets,Quick Assets/Current Liability,Cash/Current Liability,Current Liability to Assets,Operating Funds to Liability,Inventory/Working Capital,Inventory/Current Liability,Current Liabilities/Liability,Working Capital/Equity,Current Liabilities/Equity,Long-term Liability to Current Assets,Retained Earnings to Total Assets,Total income/Total expense,Total expense/Assets,Current Asset Turnover Rate,Quick Asset Turnover Rate,Working capitcal Turnover Rate,Cash Turnover Rate,Cash Flow to Sales,Fixed Assets to Assets,Current Liability to Liability,Current Liability to Equity,Equity to Long-term Liability,Cash Flow to Total Assets,Cash Flow to Liability,CFO to Assets,Cash Flow to Equity,Current Liability to Current Assets,Liability-Assets Flag,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
0,1,0.370594,0.424389,0.405750,0.601457,0.601457,0.998969,0.796887,0.808809,0.302646,0.780985,1.256969e-04,0.000000e+00,0.458143,7.250725e-04,0.000000,0.147950,0.147950,0.147950,0.169141,0.311664,0.017560,0.095921,0.138736,0.022102,0.848195,0.688979,0.688979,0.217535,4.980000e+09,0.000327,0.263100,0.363725,0.002259,0.001208,0.629951,0.021266,0.207576,0.792424,0.005024,0.390284,0.006479,0.095885,0.137757,0.398036,0.086957,0.001814,0.003487,1.820926e-04,1.165007e-04,0.032903,0.034164,0.392913,0.037135,0.672775,0.166673,0.190643,0.004094,0.001997,1.473360e-04,0.147308,0.334015,0.276920,0.001036,0.676269,0.721275,0.339077,2.559237e-02,0.903225,0.002022,0.064856,7.010000e+08,6.550000e+09,0.593831,4.580000e+08,0.671568,0.424206,0.676269,0.339077,0.126549,0.637555,0.458609,0.520382,0.312905,0.118250,0,0.716845,0.009219,0.622879,0.601453,0.827890,0.290202,0.026601,0.564050,1,0.016469
1,1,0.464291,0.538214,0.516730,0.610235,0.610235,0.998946,0.797380,0.809301,0.303556,0.781506,2.897851e-04,0.000000e+00,0.461867,6.470647e-04,0.000000,0.182251,0.182251,0.182251,0.208944,0.318137,0.021144,0.093722,0.169918,0.022080,0.848088,0.689693,0.689702,0.217620,6.110000e+09,0.000443,0.264516,0.376709,0.006016,0.004039,0.635172,0.012502,0.171176,0.828824,0.005059,0.376760,0.005835,0.093743,0.168962,0.397725,0.064468,0.001286,0.004917,9

In [ ]:
data.describe()

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,Continuous interest rate (after tax),Operating Expense Rate,Research and development expense rate,Cash flow rate,Interest-bearing debt interest rate,Tax rate (A),Net Value Per Share (B),Net Value Per Share (A),Net Value Per Share (C),Persistent EPS in the Last Four Seasons,Cash Flow Per Share,Revenue Per Share (Yuan ¥),Operating Profit Per Share (Yuan ¥),Per Share Net profit before tax (Yuan ¥),Realized Sales Gross Profit Growth Rate,Operating Profit Growth Rate,After-tax Net Profit Growth Rate,Regular Net Profit Growth Rate,Continuous Net Profit Growth Rate,Total Asset Growth Rate,Net Value Growth Rate,Total Asset Return Growth Rate Ratio,Cash Reinvestment %,Current Ratio,Quick Ratio,Interest Expense Ratio,Total debt/Total net worth,Debt ratio %,Net worth/Assets,Long-term fund suitability ratio (A),Borrowing dependency,Contingent liabilities/Net worth,Operating profit/Paid-in capital,Net profit before tax/Paid-in capital,Inventory and accounts receivable/Net value,Total Asset Turnover,Accounts Receivable Turnover,Average Collection Days,Inventory Turnover Rate (times),Fixed Assets Turnover Frequency,Net Worth Turnover Rate (times),Revenue per person,Operating profit per person,Allocation rate per person,Working Capital to Total Assets,Quick Assets/Total Assets,Current Assets/Total Assets,Cash/Total Assets,Quick Assets/Current Liability,Cash/Current Liability,Current Liability to Assets,Operating Funds to Liability,Inventory/Working Capital,Inventory/Current Liability,Current Liabilities/Liability,Working Capital/Equity,Current Liabilities/Equity,Long-term Liability to Current Assets,Retained Earnings to Total Assets,Total income/Total expense,Total expense/Assets,Current Asset Turnover Rate,Quick Asset Turnover Rate,Working capitcal Turnover Rate,Cash Turnover Rate,Cash Flow to Sales,Fixed Assets to Assets,Current Liability to Liability,Current Liability to Equity,Equity to Long-term Liability,Cash Flow to Total Assets,Cash Flow to Liability,CFO to Assets,Cash Flow to Equity,Current Liability to Current Assets,Liability-Assets Flag,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
count,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6.819000e+03,6.819000e+03,6819.000000,6.819000e+03,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6.819000e+03,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6.819000e+03,6.819000e+03,6819.000000,6819.000000,6.819000e+03,6.819000e+03,6819.000000,6.819000e+03,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6.819000e+03,6.819000e+03,6.819000e+03,6.819000e+03,6819.000000,6.819000e+03,6819.000000,6.819000e+03,6819.000000,6819.000000,6819.000000,6819.000000,6.819000e+03,6.819000e+03,6819.000000,6819.000000,6819.000000,6.819000e+03,6819.000000,6819.000000,6819.000000,6.819000e+03,6819.000000,6819.000000,6819.000000,6.819000e+03,6.819000e+03,6819.000000,6.819000e+03,6819.000000,6.819000e+03,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6.819000e+03,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.0,6819.000000
mean,0.032263,0.505180,0.558625,0.553589,0.607948,0.607929,0.998755,0.797190,0.809084,0.303623,0.781381,1.995347e+09,1.950427e+09,0.467431,1.644801e+07,0.11

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6819 entries, 0 to 6818
Data columns (total 96 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   Bankrupt?                                                 6819 non-null   int64  
 1    ROA(C) before interest and depreciation before interest  6819 non-null   float64
 2    ROA(A) before interest and % after tax                   6819 non-null   float64
 3    ROA(B) before interest and depreciation after tax        6819 non-null   float64
 4    Operating Gross Margin                                   6819 non-null   float64
 5    Realized Sales Gross Margin                              6819 non-null   float64
 6    Operating Profit Rate                                    6819 non-null   float64
 7    Pre-tax net Interest Rate                                6819 non-null   float64
 8    After-tax net Int

In [ ]:
{column: len(data[column].unique()) for column in data.columns}

{' Accounts Receivable Turnover': 1593,
 ' After-tax Net Profit Growth Rate': 6246,
 ' After-tax net Interest Rate': 3604,
 ' Allocation rate per person': 6768,
 ' Average Collection Days': 5451,
 ' Borrowing dependency': 4338,
 ' CFO to Assets': 6819,
 ' Cash Flow Per Share': 1545,
 ' Cash Flow to Equity': 6819,
 ' Cash Flow to Liability': 6819,
 ' Cash Flow to Sales': 6819,
 ' Cash Flow to Total Assets': 6819,
 ' Cash Reinvestment %': 3599,
 ' Cash Turnover Rate': 4023,
 ' Cash flow rate': 5557,
 ' Cash/Current Liability': 6816,
 ' Cash/Total Assets': 6819,
 ' Contingent liabilities/Net worth': 1855,
 ' Continuous Net Profit Growth Rate': 6270,
 ' Continuous interest rate (after tax)': 3617,
 ' Current Asset Turnover Rate': 6261,
 ' Current Assets/Total Assets': 6819,
 ' Current Liabilities/Equity': 6819,
 ' Current Liabilities/Liability': 6627,
 ' Current Liability to Assets': 6819,
 ' Current Liability to Current Assets': 6819,
 ' Current Liability to Equity': 6819,
 ' Current Liab

**Preprocessing**

In [ ]:
def preprocess_inputs(df):
  df = df.copy()

  # Drop single-value column
  df = df.drop(' Net Income Flag', axis=1)

  # Split df into X and y
  y = df['Bankrupt?']
  X = df.drop('Bankrupt?', axis=1)

  # Train-test split
  X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)

  # Scale X
  scaler = StandardScaler()
  scaler.fit(X_train)
  X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
  X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)
  return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = preprocess_inputs(data)

In [ ]:
X_train

,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,Continuous interest rate (after tax),Operating Expense Rate,Research and development expense rate,Cash flow rate,Interest-bearing debt interest rate,Tax rate (A),Net Value Per Share (B),Net Value Per Share (A),Net Value Per Share (C),Persistent EPS in the Last Four Seasons,Cash Flow Per Share,Revenue Per Share (Yuan ¥),Operating Profit Per Share (Yuan ¥),Per Share Net profit before tax (Yuan ¥),Realized Sales Gross Profit Growth Rate,Operating Profit Growth Rate,After-tax Net Profit Growth Rate,Regular Net Profit Growth Rate,Continuous Net Profit Growth Rate,Total Asset Growth Rate,Net Value Growth Rate,Total Asset Return Growth Rate Ratio,Cash Reinvestment %,Current Ratio,Quick Ratio,Interest Expense Ratio,Total debt/Total net worth,Debt ratio %,Net worth/Assets,Long-term fund suitability ratio (A),Borrowing dependency,Contingent liabilities/Net worth,Operating profit/Paid-in capital,Net profit before tax/Paid-in capital,Inventory and accounts receivable/Net value,Total Asset Turnover,Accounts Receivable Turnover,Average Collection Days,Inventory Turnover Rate (times),Fixed Assets Turnover Frequency,Net Worth Turnover Rate (times),Revenue per person,Operating profit per person,Allocation rate per person,Working Capital to Total Assets,Quick Assets/Total Assets,Current Assets/Total Assets,Cash/Total Assets,Quick Assets/Current Liability,Cash/Current Liability,Current Liability to Assets,Operating Funds to Liability,Inventory/Working Capital,Inventory/Current Liability,Current Liabilities/Liability,Working Capital/Equity,Current Liabilities/Equity,Long-term Liability to Current Assets,Retained Earnings to Total Assets,Total income/Total expense,Total expense/Assets,Current Asset Turnover Rate,Quick Asset Turnover Rate,Working capitcal Turnover Rate,Cash Turnover Rate,Cash Flow to Sales,Fixed Assets to Assets,Current Liability to Liability,Current Liability to Equity,Equity to Long-term Liability,Cash Flow to Total Assets,Cash Flow to Liability,CFO to Assets,Cash Flow to Equity,Current Liability to Current Assets,Liability-Assets Flag,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Equity to Liability
3392,-7.065972,-7.189713,-7.577832,-0.353178,-0.352216,-27.422519,-9.852867,-9.473508,0.174600,-17.488805,-0.623336,-0.750627,-3.409234,-0.146024,-0.831378,-1.684936,-1.679626,-1.680602,-2.210939,-1.269358,-0.023643,-2.383154,-1.803000,-0.072283,-0.149416,-0.092698,-0.093621,-0.017514,-1.071132,-0.123720,-0.358472,-6.762465,-0.014476,-0.029836,-0.031652,-0.025154,-0.245936,0.245936,-0.118704,-0.292121,-0.045435,-2.398334,-1.930971,-0.554824,-1.366000,-0.04243,-0.038594,0.886224,0.736406,-0.842540,-0.020349,-0.918259,-0.039086,0.776376,1.652612,0.989620,-0.369080,-0.014476,-0.074892,0.334264,-4.717374,-0.033918,-0.090308,1.161424,0.227750,-0.066147,-0.092181,-6.779749,-0.045371,8.415101,-0.423261,-0.643034,0.806322,-0.842942,-1.475297,-0.161486,1.161424,-0.066147,-0.219187,-5.838952,-2.424303,-8.876742,-3.764293,-0.301941,-0.032383,-9.998552,4.692638,-0.014135,-0.353225,-1.832779,-0.140534,-0.046856,-0.002980,-0.193823
2755,0.405365,0.354931,0.313557,0.203699,0.200630,0.105921,0.077435,0.062927,0.047214,0.099756,-0.623336,1.316196,0.168498,-0.146024,0.987475,-0.101405,-0.100452,-0.101962,0.111816,0.190816,-0.023643,-0.097101,0.136276,-0.080535,-0.032348,0.008982,0.007273,-0.008501,0.377593,-0.034893,-0.058556,0.354458,-0.014476,-0.029836,-0.006998,-0.025154,-0.432271,0.432271,-0.119722,-0.156458,-0.017916,-0.092838,0.172557,-0.268783,-0.3

In [ ]:
y_train

3392    0
2755    0
4442    0
4267    0
4912    0
       ..
905     0
5192    0
3980    0
235     1
5157    0
Name: Bankrupt?, Length: 4773, dtype: int64

**Training original data**

In [ ]:
original_models = {
    'Logistic Regression': LogisticRegression(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    'Support Vector Machine (Linear Kernel)': LinearSVC(),
    'Support Vector Machine (RBF Kernel)': SVC(),
    'Neural Network': MLPClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier()
}

for name, model in original_models.items():
  model.fit(X_train, y_train)
  print(name + ' trained.')

Logistic Regression trained.
K-Nearest Neighbors trained.
Decision Tree trained.
Support Vector Machine (Linear Kernel) trained.
Support Vector Machine (RBF Kernel) trained.
Neural Network trained.
Random Forest trained.
Gradient Boosting trained.


In [ ]:
original_results = []

for name, model in original_models.items():
  result = model.score(X_test, y_test)
  original_results.append(result)
  print(name + ': {:.2f}%'.format(result * 100))

Logistic Regression: 96.43%
K-Nearest Neighbors: 96.48%
Decision Tree: 94.97%
Support Vector Machine (Linear Kernel): 96.19%
Support Vector Machine (RBF Kernel): 96.58%
Neural Network: 96.09%
Random Forest: 96.43%
Gradient Boosting: 96.04%


In [ ]:
y_test.value_counts() / len(y_test)

0    0.965298
1    0.034702
Name: Bankrupt?, dtype: float64

As we can see that only one model is Support Vector Machine (RBF Kernel) that has the score (96.58%) is nearly higher than the score even if we guess all zero in y_test dataset (96.53%).

**Dimensionality Reduction (PCA)**

In [ ]:
n_components = 94

pca = PCA(n_components=n_components)
pca.fit(X_train)

X_train_reduced = pd.DataFrame(pca.transform(X_train), index=X_train.index, columns=['PC'+str(i) for i in range(1, n_components+1)])
X_test_reduced = pd.DataFrame(pca.transform(X_test), index=X_test.index, columns=['PC'+str(i) for i in range(1, n_components+1)])


In [ ]:
X_train_reduced

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20,PC21,PC22,PC23,PC24,PC25,PC26,PC27,PC28,PC29,PC30,PC31,PC32,PC33,PC34,PC35,PC36,PC37,PC38,PC39,PC40,PC41,PC42,PC43,PC44,PC45,PC46,PC47,PC48,PC49,PC50,PC51,PC52,PC53,PC54,PC55,PC56,PC57,PC58,PC59,PC60,PC61,PC62,PC63,PC64,PC65,PC66,PC67,PC68,PC69,PC70,PC71,PC72,PC73,PC74,PC75,PC76,PC77,PC78,PC79,PC80,PC81,PC82,PC83,PC84,PC85,PC86,PC87,PC88,PC89,PC90,PC91,PC92,PC93,PC94
3392,21.396971,-7.427836,-10.384790,16.385044,8.410605,14.815803,-1.072838,0.033075,2.402385,-2.696531,-4.214135,6.352764,-11.027381,1.913793,1.606729,1.729124,0.570398,-0.789446,-0.892282,-0.320629,-1.797002,-2.299265,0.158909,-2.584296,-2.901013,1.291582,0.719369,0.071927,-1.077647,1.268982,0.234510,-1.286139,-2.170726,0.026729,0.659986,1.623508,-0.299143,-1.253455,0.089233,1.378367,-3.465337,-0.325044,-5.566624,-0.114496,-2.370308,-1.374702,0.387789,0.008084,-0.623874,2.009423,1.666888,-6.253381,-5.552162,-5.670375,13.485811,5.164796,2.414657,0.248695,5.763678,1.951909,-0.096881,-1.057261,2.146485,-1.081045,7.166036,1.682672,0.923173,1.780433,-1.115230,-0.843055,-0.433020,1.470648,-0.138616,-0.440672,-1.108144,0.010372,0.930639,-0.632512,-0.124990,-0.251547,-1.437326,0.054612,0.302335,-0.058173,-0.031229,0.057488,0.262348,0.005251,1.425501e-05,8.958819e-06,-7.364276e-10,5.688374e-15,-7.500056e-15,2.320156e-15
2755,-0.537250,-0.632416,1.522062,0.059505,-0.892918,0.115449,0.166554,-0.332114,-0.642663,-0.422476,0.569907,-0.704901,0.165873,-0.239947,0.391633,0.419027,-0.332370,-0.234684,0.223903,0.538678,-1.338388,0.872265,1.142810,0.173128,-0.165659,0.182387,-0.318347,-0.268019,-0.151064,0.371043,-0.610176,0.540534,-0.316358,0.052687,-0.047715,0.225196,-1.077294,0.739121,-0.639626,-0.100860,0.721593,-1.240560,-0.501506,0.063968,0.379138,-0.043415,-0.135799,-0.568019,0.136798,-0.411147,0.113967,-0.007154,0.158042,0.227862,0.527669,-0.189042,0.114654,0.013197,-0.045299,-0.071416,-0.107512,0.158320,-0.154621,-0.242867,0.027840,-0.131080,0.127882,0.022143,0.056348,0.011610,-0.015061,0.001409,-0.010412,0.032839,-0.003984,-0.028037,-0.017289,0.021823,0.001008,0.013471,0.007356,-0.012186,-0.001268,0.001103,0.000554,-0.000496,0.001396,0.000096,-7.010621e-06,-3.457161e-05,-8.154396e-11,3.826907e-16,1.533091e-16,2.066226e-16
4442,-3.777733,-0.482875,1.127331,0.334114,1.277027,0.359205,0.872919,-0.504780,-1.627089,-1.011487,1.137812,-0.990688,0.020866,0.753910,0.551596,-0.209501,0.195654,0.323243,-0.338553,0.361258,-0.945430,0.968478,0.652160,0.036588,0.389052,0.179218,0.169331,-0.071738,0.091093,-0.075175,-0.328946,0.002957,0.206092,0.648976,-0.053190,-0.402783,0.213547,0.060351,0.461387,-0.270898,-0.165188,1.563078,0.335242,0.200471,-0.000799,-0.080850,0.157284,-0.297051,0.606349,-0.728476,0.456325,-0.393636,0.184266,-0.454713,0.214520,0.235545,-0.154430,-0.207144,-0.302312,-0.311534,0.309213,-0.333500,0.573638,0.242300,-0.057151,0.164958,-0.088129,-0.135636,-0.132184,0.064894,-0.124100,-0.120966,0.026133,0.046278,-0.017616,0.018570,-0.006252,-0.033432,0.012967,0.026627,-0.027596,-0.016177,0.008452,0.005571,-0.011204,0.007751,0.003282,0.000274,-1.623794e-04,3.562151e-05,2.126603e-11,1.063157e-16,-2.639799e-16,1.075285e-16
4267,3.117700,-0.669615,1.191915,-0.644932,-1.861750,0.650279,0.850108,-0.755383,-0.427210,-0.891521,2.446697,0.372685,-0.289336,-0.837916,-0.458419,-1.117063,1.562364,-0.621951,0.089918,-0.036900,-1.141743,1.467604,0.620103,-0.020184,-0.798211,1.344014,-1.028388,-0.545923,0.273551,0.780606,-1.704963,-0.071611,-0.486957,0.147964,-0.206660,1.481683,-0.057097,0.395285,-0.809322,-0.551451,0.363974,-1.241436,-1.223641,0.086683,0.756062,0.506555,-0.233691,1.014928,-0.963541,1.120362,-1.358330,0.992346,-0.710175,0.578414,0.873999,-1.239706,0.049998,0.099915,-0.651922,1.418889,-0.041258,-0.519515,-0.045136,-0.426192,-0.061291,0.235479,-0.078614,0.132220,-0.025381,0.027091,0.025570,-0.062664,-0.209238,-0.071025,0.006483,0.108301,0.089400,0.040665,-0.010364,0.046168,0.02160

In [ ]:
X_train_reduced.describe()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20,PC21,PC22,PC23,PC24,PC25,PC26,PC27,PC28,PC29,PC30,PC31,PC32,PC33,PC34,PC35,PC36,PC37,PC38,PC39,PC40,PC41,PC42,PC43,PC44,PC45,PC46,PC47,PC48,PC49,PC50,PC51,PC52,PC53,PC54,PC55,PC56,PC57,PC58,PC59,PC60,PC61,PC62,PC63,PC64,PC65,PC66,PC67,PC68,PC69,PC70,PC71,PC72,PC73,PC74,PC75,PC76,PC77,PC78,PC79,PC80,PC81,PC82,PC83,PC84,PC85,PC86,PC87,PC88,PC89,PC90,PC91,PC92,PC93,PC94
count,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03,4.773000e+03
mean,-3.154122e-17,7.885305e-17,2.093444e-18,1.586162e-16,-4.117106e-18,-3.354162e-17,-7.350314e-17,1.372369e-17,2.321397e-17,3.583859e-17,-5.787209e-17,1.237458e-17,-3.070384e-18,-5.303391e-18,-2.246963e-17,-1.221176e-17,-1.525888e-17,5.375353e-18,1.240947e-17,5.570887e-18,-6.378025e-17,-8.285385e-17,-2.795910e-17,-2.378676e-17,2.551675e-17,-1.321196e-17,-2.466193e-17,9.554245e-18,-1.721276e-17,-1.674755e-17,1.279327e-17,9.548430e-18,-2.570284e-17,-1.504953e-17,1.411912e-17,-2.738922e-17,-8.583120e-18,-7.489877e-18,-2.438281e-17,2.686586e-18,-2.186486e-18,-2.530741e-17,1.472389e-17,1.905615e-17,-3.256468e-18,-8.594750e-18,-1.744536e-18,3.605375e-18,-3.884501e-18,1.263044e-17,5.524366e-19,1.979467e-17,-5.059156e-18,1.924805e-18,-8.559859e-18,2.244637e-18,1.261300e-17,1.779427e-18,-2.105074e-18,7.396835e-18,3.791459e-18,-3.829258e-18,1.129878e-17,4.907963e-18,-1.318288e-17,1.084520e-17,-2.134731e-17,-1.870434e-17,-6.446062e-18,1.775357e-17,5.538903e-18,-1.249670e-17,2.512133e-18,-2.180671e-19,-1.267697e-17,9.917690e-18,-2.634250e-18,1.354923e-17,-4.518713e-18,-9.182077e-18,5.221979e-18,-1.127843e-17,-4.378423e-18,6.893100e-18,-6.339936e-18,1.145579e-17,4.233136e-18,6.820774e-18,8.224746e-18,1.016780e-19,1.690083e-17,2.226799e-17,5.631357e-18,-1.542189e-17
std,3.608307e+00,2.666728e+00,2.301153e+00,2.240383e+00,2.170172e+00,1.802285e+00,1.736072e+00,1.499769e+00,1.485012e+00,1.445549e+00,1.394536e+00,1.338177e+00,1.329033e+00,1.276697e+00,1.223592e+00,1.164099e+00,1.158493e+00,1.121536e+00,1.114318e+00,1.103978e+00,1.094967e+00,1.070406e+00,1.042191e+00,1.022780e+00,1.016503e+00,1.012519e+00,1.006838e+00,1.002407e+00,1.000604e+00,9.944115e-01,9.885120e-01,9.864065e-01,9.785258e-01,9.775492e-01,9.746456e-01,9.546814e-01,9.440345e-01,9.316446e-01,9.212385e-01,9.066758e-01,8.961778e-01,8.835767e-01,8.560177e-01,8.291014e-01,8.157149e-01,7.813326e-01,7.752720e-01,7.584829e-01,7.198249e-01,7.079448e-01,6.666836e-01,6.555776e-01,6.423832e-01,6.111805e-01,5.982282e-01,5.655035e-01,5.335609e-01,5.149912e-01,4.839961e-01,4.680639e-01,4.465116e-01,4.165755e-01,4.096256e-01,4.001292e-01,3.666637e-01,3.642006e-01,3.013836e-01,2.613791e-01,2.450246e-01,2.305475e-01,2.250341e-01,2.043648e-01,1.807070e-01,1.737111e-01,1.473727e-01,1.407645e-01,1.189050e-01,1.111018e-01,1.005816e-01,8.79

In [ ]:
# Show each variance of columns
pca.explained_variance_

array([1.30198766e+01, 7.11143711e+00, 5.29530377e+00, 5.01931524e+00,
       4.70964778e+00, 3.24822970e+00, 3.01394560e+00, 2.24930645e+00,
       2.20526006e+00, 2.08961284e+00, 1.94472927e+00, 1.79071697e+00,
       1.76632868e+00, 1.62995592e+00, 1.49717652e+00, 1.35512560e+00,
       1.34210617e+00, 1.25784407e+00, 1.24170572e+00, 1.21876845e+00,
       1.19895366e+00, 1.14576831e+00, 1.08616155e+00, 1.04607929e+00,
       1.03327812e+00, 1.02519478e+00, 1.01372255e+00, 1.00481891e+00,
       1.00120774e+00, 9.88854216e-01, 9.77155955e-01, 9.72997752e-01,
       9.57512828e-01, 9.55602468e-01, 9.49934087e-01, 9.11416486e-01,
       8.91201118e-01, 8.67961641e-01, 8.48680373e-01, 8.22060930e-01,
       8.03134635e-01, 7.80707837e-01, 7.32766300e-01, 6.87409209e-01,
       6.65390757e-01, 6.10480629e-01, 6.01046696e-01, 5.75296277e-01,
       5.18147916e-01, 5.01185780e-01, 4.44467008e-01, 4.29782002e-01,
       4.12656177e-01, 3.73541589e-01, 3.57877010e-01, 3.19794169e-01,
      

In [ ]:
# Variance ratio of components
pca.explained_variance_ratio_

array([1.38480306e-01, 7.56377360e-02, 5.63212164e-02, 5.33857833e-02,
       5.00921388e-02, 3.45483953e-02, 3.20565334e-02, 2.39237787e-02,
       2.34552982e-02, 2.22252664e-02, 2.06842747e-02, 1.90461893e-02,
       1.87867938e-02, 1.73363237e-02, 1.59240728e-02, 1.44132094e-02,
       1.42747339e-02, 1.33785163e-02, 1.32068677e-02, 1.29629054e-02,
       1.27521538e-02, 1.21864708e-02, 1.15524893e-02, 1.11261716e-02,
       1.09900174e-02, 1.09040424e-02, 1.07820231e-02, 1.06873233e-02,
       1.06489146e-02, 1.05175217e-02, 1.03930982e-02, 1.03488713e-02,
       1.01841725e-02, 1.01638538e-02, 1.01035645e-02, 9.69388866e-03,
       9.47887660e-03, 9.23169992e-03, 9.02662303e-03, 8.74349680e-03,
       8.54219542e-03, 8.30366244e-03, 7.79375295e-03, 7.31133180e-03,
       7.07714201e-03, 6.49311410e-03, 6.39277415e-03, 6.11889091e-03,
       5.51105699e-03, 5.33064655e-03, 4.72738177e-03, 4.57119104e-03,
       4.38903958e-03, 3.97301413e-03, 3.80640458e-03, 3.40135285e-03,
      

In [ ]:
fig = px.bar(
    x=['PC'+str(i) for i in range(1, n_components+1)],
    y=pca.explained_variance_ratio_,
    labels={'x': 'Principal Component', 'y': 'Variance Ratio'},
    color=pca.explained_variance_ratio_,
    color_continuous_scale=[(0, 'lightblue'), (1, 'darkblue')],
    title='Proportion of Variance in Principal Components'
)
fig.show()

Have a look at the graph above, we realize that the final principal components have very little information because the variance is so small and we can drop off these lower columns to reduce dimensions.

Just remain only 10 principal components has the highest variance.

In [ ]:
n_components = 10

pca = PCA(n_components=n_components)
pca.fit(X_train)

X_train_reduced = pd.DataFrame(pca.transform(X_train), index=X_train.index, columns=['PC'+str(i) for i in range(1, n_components+1)])
X_test_reduced = pd.DataFrame(pca.transform(X_test), index=X_test.index, columns=['PC'+str(i) for i in range(1, n_components+1)])


In [ ]:
fig = px.bar(
    x=['PC'+str(i) for i in range(1, n_components+1)],
    y=pca.explained_variance_ratio_,
    labels={'x': 'Principal Component', 'y': 'Variance Ratio'},
    color=pca.explained_variance_ratio_,
    color_continuous_scale=[(0, 'lightblue'), (1, 'darkblue')],
    title='Proportion of Variance in Principal Components'
)
fig.show()

**Training (Reduced Data)**

In [ ]:
reduced_models = {
    'Logistic Regression': LogisticRegression(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    'Support Vector Machine (Linear Kernel)': LinearSVC(),
    'Support Vector Machine (RBF Kernel)': SVC(),
    'Neural Network': MLPClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier()
}

for name, model in reduced_models.items():
  model.fit(X_train_reduced, y_train)
  print(name + ' trained.')

Logistic Regression trained.
K-Nearest Neighbors trained.
Decision Tree trained.
Support Vector Machine (Linear Kernel) trained.
Support Vector Machine (RBF Kernel) trained.
Neural Network trained.
Random Forest trained.
Gradient Boosting trained.


In [ ]:
reduced_results = []

for name, model in reduced_models.items():
  result = model.score(X_test_reduced, y_test)
  reduced_results.append(result)
  print(name + ': {:.2f}%'.format(result * 100))

Logistic Regression: 96.24%
K-Nearest Neighbors: 96.43%
Decision Tree: 95.16%
Support Vector Machine (Linear Kernel): 96.43%
Support Vector Machine (RBF Kernel): 96.63%
Neural Network: 96.24%
Random Forest: 96.58%
Gradient Boosting: 96.19%


**Change in Performance After PCA**

In [ ]:
fig = px.bar(
    x=np.subtract(reduced_results, original_results),
    y=reduced_models.keys(),
    orientation='h',
    labels={'x': 'Change in Performance', 'y': 'Model'},
    color=np.subtract(reduced_results, original_results),
    color_continuous_scale=[(0, 'red'), (1, 'blue'),],
    title='Change in Model Performance After Dimensionality Reduction'
)
fig.show()